In [2]:
%pip install gme
%pip install gegravity



Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 48 kB 1.8 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [2]:
__Author__ = "Peter Herman"
__Project__ = "gegravity"
__Created__ = "March 01, 2021"
__Description__ = """A demonstration of the OneSectorGE model and module"""
#change dataset location, year 2006/2021, counterfactual
# ---
# Load Packages
# ---
import gegravity as ge
import pandas as pd
# Increase number of columns printed for a pandas DataFrame
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)
import gme as gme


In [8]:
# ----
# Load some gravity data
# ----
#gravity_data_location = "square from stata for python.csv"
#gravity_data_location = "sample_gravity_data.csv"
#gravity_data_location = "twentysevenherman.csv"
gravity_data_location = "hermantrade2.csv"

grav_data = pd.read_csv(gravity_data_location)
print(grav_data.head())


   simpleaverage exporter importer  reporter_prevalence_score  partner_prevalence_score       imports         trade  exports_reporter  imports_reporter  exports_partner  imports_partner             Y  production_partner             E  contiguity  common_language  colony  comcol       dist       distw  year    lndist  international  pta  lntariff  ln_importer_prevalence  ln_exporter_prevalence   partner_gdp  reporter_gdp  ln_exporter_gdp  ln_importer_gdp
0            NaN      ARE      ARE                        NaN                       NaN           NaN  2.620000e+11      1.380000e+11      1.440000e+11     1.380000e+11     1.440000e+11  4.000000e+11        4.000000e+11  4.060000e+11           0                0       0       0    108.789     87.0386  2021  4.689410              0    0       NaN                     NaN                     NaN  4.000000e+11  4.000000e+11         26.71473        26.714730
1        4.28000      ARE      AUS                        3.4                       

In [9]:


# ----
# Prepare data and econometric inputs for GE Model
# ----

# Define GME Estimation Data
gme_data = gme.EstimationData(grav_data, # Dataset
                              imp_var_name="importer", # Importer column name
                              exp_var_name="exporter", # Exporter column name
                              year_var_name = "year",  # Year column name
                              trade_var_name="trade")  # Trade column name
# Create Gravity Model
gme_model = gme.EstimationModel(gme_data, # Specify data to use
                                lhs_var="trade",                               # dependent, "left hand side" variable
                                rhs_var=["pta","contiguity","common_language", # independent variables
                                         "lndist","international", "comcol", "ln_exporter_gdp", "ln_importer_gdp"],
                                fixed_effects=[["exporter"],["importer"]])     # Fixed effects to use
#"lntariff", "ln_exporter_prevalence", "ln_importer_prevalence"
# Estimate gravity model with PPML
gme_model.estimate()
# Print econometric results table
print(gme_model.results_dict['all'].summary())




Estimation began at 06:26 PM  on Feb 03, 2024
Omitted Regressors: ['exporter_fe_ZAF', 'importer_fe_USA', 'importer_fe_ZAF']
Estimation completed at 06:26 PM  on Feb 03, 2024
                 Generalized Linear Model Regression Results                  
Dep. Variable:                  trade   No. Observations:                  729
Model:                            GLM   Df Residuals:                      670
Model Family:                 Poisson   Df Model:                           58
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.6062e+12
Date:                Sat, 03 Feb 2024   Deviance:                   9.2125e+12
Time:                        18:26:18   Pearson chi2:                 1.25e+13
No. Iterations:                    19   Pseudo R-squ. (CS):              1.000
Covariance Type:                  HC1                                         
                      coef    std er

/Users/chetanyabhan/opt/anaconda3/lib/python3.9/site-packages/gme/estimate/_ppml_estimation_and_diagnostics.py:306: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diagnostics = overfit_column.append(exclusion_column)


In [10]:

# ----
# Define a GE model
# ----

# Define GE model
ge_model = ge.OneSectorGE(gme_model,                   # gme gravity model
                       year = "2021",               # Year to use for model
                       expend_var_name = "E",       # Expenditure column name
                       output_var_name = "Y",       # Output column name
                       reference_importer = "IND",  # Reference importer
                       sigma = 5)                   # Elasticity of substitution




In [11]:


# ----
# Diagnose model solvability
# ----

# The following commands are not required to define or solve the GE model but can help diagnose issues that arise if
# the model fails to solve.

##
# Check inputs
##

# Test that the model system of equations is computable from the supplied data and parameters
test_diagnostics = ge_model.test_baseline_mr_function()
# See what is returned:
#print(test_diagnostics.keys())
# Check the values of the model parameters computed from the baseline data, which should be numeric with no missing
# values
input_params = test_diagnostics['mr_params']
# Check one set of parameters, for example:
print(input_params['cost_exp_shr'])


[[8.99943226e+09 4.86990410e+07 1.48386004e+07 7.56695274e+07
  2.80368100e+06 2.99934562e+09 1.51373855e+07 2.19916198e+09
  1.83753991e+06 4.04390121e+07 2.28213767e+09 1.82978534e+06
  4.12338090e+05 5.02570263e+08 2.47824936e+05 2.14248284e+06
  1.17996686e+05 1.34386093e+07 1.05218088e+06 2.81958908e+07
  3.10771454e+07 6.02018400e+08 1.43236083e+07 6.46474074e+07
  2.57935998e+10 3.18980672e+07 1.11697833e+09]
 [4.16014537e+07 5.60880967e+09 2.78676387e+07 7.96724699e+07
  5.84012818e+06 3.78621249e+09 2.51860930e+07 1.52565056e+09
  3.27296904e+06 7.97021211e+07 4.00575037e+09 2.08918809e+06
  5.87601054e+05 7.48995012e+08 3.07279922e+05 3.87485739e+06
  1.17246106e+05 4.43686058e+07 5.64177744e+06 2.36445413e+07
  4.59641044e+07 3.70360139e+08 4.05170334e+07 8.09330799e+07
  2.64752677e+10 3.25487737e+07 1.06060334e+09]
 [2.41311770e+07 5.30514967e+07 7.63283223e+08 6.07911131e+07
  2.11367192e+06 9.66587905e+08 1.15128637e+07 1.41508269e+09
  1.40294116e+06 5.42907728e+07 2.91

In [12]:


##
# Check scaling of outward multilateral resistances (OMRs)
##

# Check for OMR rescale factors that results in convergence
rescale_eval = ge_model.check_omr_rescale(omr_rescale_range=3)
print(rescale_eval)
# From the tests, it looks like 10, 100, and 1000 are good candidate rescale factors based on the fact that
# the model solves (i.e. converges) and all three produce consistent solutions for the reference importer's
# outward multilateral resistance (OMR) terms (2.918).




Trying OMR rescale factor of 0.001
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 0.01
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 0.1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 10
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 100
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1000
Solving for baseline MRs...
The solution converged.
   omr_rescale omr_rescale (alt format)  solved                  message  max_func_value  mean_func_value  reference_importer_omr
0        0.001                    10^-3    True  The solution converged.    2.809676e-09    -5.056910e-11                0.002200
1        0.010                    10^-2    True  The solution converged.    2.627274e-10    -3.107070e-12                0.002200
2      

In [13]:


# ---
# Solve baseline and experiment GE model
# ---

##
# Solve the baseline model
##
ge_model.build_baseline(omr_rescale=100)
# Examine the solutions for the baselin multilateral resistances
print(ge_model.baseline_mr.head(12))


Solving for baseline MRs...
The solution converged.
         baseline omr  baseline imr
country                            
ARE          0.002253      0.999993
AUS          0.002414      0.999993
BGD          0.002594      0.999993
CAN          0.002007      0.999993
CHL          0.002563      0.999993
CHN          0.002059      0.999993
COL          0.002307      0.999993
DEU          0.001915      0.999993
ECU          0.002565      0.999993
IDN          0.002362      0.999993
IND          0.002093      1.000000
JPN          0.001949      0.999993


In [14]:

##  
# Define the counterfactual experiment
##
# Create a copy of the baseline data
exp_data = ge_model.baseline_data.copy()

# Modify the copied data to reflect a counterfactual experiment in India (IND) leaves
# APTA (pta)
exp_data.loc[(exp_data["importer"] == "IND") & (exp_data["exporter"] == "IND"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "IND") & (exp_data["exporter"] == "IND"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "BGD") & (exp_data["exporter"] == "IND"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "IND") & (exp_data["exporter"] == "BGD"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "CHN") & (exp_data["exporter"] == "IND"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "IND") & (exp_data["exporter"] == "CHN"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "MNG") & (exp_data["exporter"] == "IND"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "IND") & (exp_data["exporter"] == "MNG"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "KOR") & (exp_data["exporter"] == "IND"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "IND") & (exp_data["exporter"] == "KOR"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "LKA") & (exp_data["exporter"] == "IND"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "IND") & (exp_data["exporter"] == "LKA"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "LAO") & (exp_data["exporter"] == "IND"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "IND") & (exp_data["exporter"] == "LAO"), "pta"] = 1
## Define the experiment within the GE model
ge_model.define_experiment(exp_data)
# Examine the baseline and counterfactual trade costs
print(ge_model.bilateral_costs.head())


                   baseline trade cost  experiment trade cost  trade cost change (%)
exporter importer                                                                   
ARE      ARE              9.061364e+11           9.061364e+11                    0.0
         AUS              6.613912e+09           6.613912e+09                    0.0
         BGD              2.174169e+09           2.174169e+09                    0.0
         CAN              7.752694e+09           7.752694e+09                    0.0
         CHL              1.567891e+09           1.567891e+09                    0.0


In [15]:

##
# Simulate the counterfactual model
#
ge_model.simulate()
# Examine the counterfactual trade flows predicted by the model.
print(ge_model.bilateral_trade_results.head())


Solving for conditional MRs...
The solution converged.
Solving full GE model...
The solution converged.
                   baseline modeled trade  experiment trade  trade change (percent)
exporter importer                                                                  
ARE      ARE                 6.370323e+10      1.384489e+11              117.334196
         AUS                 3.447202e+08      1.125696e+08              -67.344648
         BGD                 1.050362e+08      5.832823e+08              455.315246
         CAN                 5.356326e+08      5.706995e+08                6.546810
         CHL                 1.984608e+07      5.865592e+07              195.554151


In [17]:


# -----
# Access and Export Results
# -----

##
# Retrieve many of the different sets of model results
##
# A collection of many of the key country-level results (prices, total imports/exports, GDP, welfare, etc.)
country_results = ge_model.country_results
# The bilateral trade results
bilateral_results = ge_model.bilateral_trade_results
# A wider selection of aggregate, country-level trade results
agg_trade = ge_model.aggregate_trade_results
# country multilateral resistance (MR) terms
mr_terms = ge_model.country_mr_terms
# Get the solver diaganoistics, which is a dictionary containing many types of solver diagnostic info
solver_diagnostics = ge_model.solver_diagnostics

##
# Export results
##
# Export the results to a collection of spreadsheet (.csv) files and add trade values in levels to the outputs.
ge_model.export_results(directory="/Users/chetanyabhan/python/ECO342/",name="IND_leaves_APTA_experiment2", include_levels = True)
# It is also possible to add alternative country identifies such as full country names using the country_names argument.
# See the technical documentation for details



# -------
# Post Estimation Analysis
# -------

# Examine how the counterfactual experiment affected Indian imports from APTA
apta_share = ge_model.trade_share(importers = ['IND'],exporters = ['MNG', 'LKA', 'BGD', 'KOR', 'LAO'])
print(apta_share)


                                         description baseline modeled trade experiment trade change (percentage point)  change (%)
0  Percent of IND imports from MNG, LKA, BGD, KOR...               0.601743         0.805524                  0.203781   33.865127
1  Percent of MNG, LKA, BGD, KOR, LAO exports to IND               0.577568         1.298708                  0.721139  124.857777


In [18]:

# Print the first few rows of country-level estimated change in factory prices, GDP, and foreign exports
print(country_results[['factory gate price change (percent)', 'GDP change (percent)',
                       'foreign exports change (percent)']].head())


         factory gate price change (percent)  GDP change (percent)  foreign exports change (percent)
country                                                                                             
ARE                                -6.824363            -25.503192                        -73.994967
AUS                               -23.007371             -5.731942                        -54.444561
BGD                               -16.951741            -48.970656                        -64.305777
CAN                                 0.803472             -1.768456                        -84.519091
CHL                               -25.969212            -48.251482                        -53.915349


In [19]:

# Calculate counterfactual experiment trade based on observed levels and estimated changes in trade
levels = ge_model.calculate_levels()
print(levels.head())


          baseline observed foreign exports  experiment observed foreign exports  baseline observed foreign imports  experiment observed foreign imports  baseline observed intranational trade  experiment observed intranational trade
exporter                                                                                                                                                                                                                                
ARE                            1.435302e+11                         3.732508e+10                       1.383935e+11                         2.177460e+11                           2.620000e+11                             5.694156e+11
AUS                            3.002898e+11                         1.367983e+11                       1.820175e+11                         4.533240e+10                           5.990000e+08                             3.466896e+08
BGD                            2.731542e+10                         

In [20]:

# Calculate trade weighted shocks
# Start with bilateral level shocks
bilat_cost_shock = ge_model.trade_weighted_shock(how = 'bilateral')
print(bilat_cost_shock.head())


  exporter importer  baseline modeled trade  trade cost change (%)  weighted_cost_change
0      ARE      ARE            6.370323e+10                    0.0                   NaN
1      ARE      AUS            3.447202e+08                    0.0                   NaN
2      ARE      BGD            1.050362e+08                    0.0                   NaN
3      ARE      CAN            5.356326e+08                    0.0                   NaN
4      ARE      CHL            1.984608e+07                    0.0                   NaN


In [21]:
# Now at country-level, which summarizes the bilateral shocks
country_cost_shock  = ge_model.trade_weighted_shock(how='country', aggregations = ['mean', 'sum', 'max'])
print(country_cost_shock.head())





    weighted_cost_change                                             
                    mean      sum      max     mean      sum      max
                exporter exporter exporter importer importer importer
ARE                  NaN      0.0      NaN      NaN      0.0      NaN
AUS                  NaN      0.0      NaN      NaN      0.0      NaN
BGD                  NaN      0.0      NaN      NaN      0.0      NaN
CAN                  NaN      0.0      NaN      NaN      0.0      NaN
CHL                  NaN      0.0      NaN      NaN      0.0      NaN


In [22]:
# ----
# Create model with alternative cost estimates
# ----

# Prepare a DataFrame with the desired cost parameter values.
coeff_data = [{'var':"lndist", 'coeff':-0.3, 'ste':0.10},
              {'var':"contiguity", 'coeff':1.9, 'ste':0.48},
              {'var':"pta", 'coeff':-0.6, 'ste':0.34},
              {'var':"common_language", 'coeff':-0.3, 'ste':0.24},
              {'var':"international", 'coeff':-5.1, 'ste':1.1}]
coeff_df = pd.DataFrame(coeff_data)
print(coeff_df)



               var  coeff   ste
0           lndist   -0.3  0.10
1       contiguity    1.9  0.48
2              pta   -0.6  0.34
3  common_language   -0.3  0.24
4    international   -5.1  1.10


In [23]:
# Create a CostCoeff object from those values
cost_params = ge.CostCoeffs(estimates = coeff_df, identifier_col = 'var', coeff_col = 'coeff', stderr_col = 'ste')

# Define a new OneSector GE model using those cost parameters
alternate_costs = ge.OneSectorGE(gme_model, year = "2021",
                       expend_var_name = "E",  # Expenditure column name
                       output_var_name = "Y",  # Output column name
                       reference_importer = "IND",  # Reference importer
                       sigma = 5,
                       cost_coeff_values=cost_params)
alternate_costs.build_baseline(omr_rescale=10)

KeyError: "['comcol', 'ln_exporter_gdp', 'ln_importer_gdp'] not in index"